### Carga de las imagenes

In [3]:
from image_services import get_image_urls
from skimage import io
import solar_image_analysis as sia
import re 
from datetime import datetime

Acá esta como ejemplo como podemos generar todos los links para cualquera de las  carpertas dado un rango de fechas, el nombre de la carpeta (tipo de imagen, en que banda se tomo), indicando apropiadamente la fecha de incio y la frecuencia con la que se toman, como se muestra a continuación estan generando para tres casos


In [16]:
hmiigr_urls = get_image_urls('01/01/2023 00:00:00', '02/01/2023', image_type = 'hmiigr' ,frequency='90T')
eit171_urls = get_image_urls('01/01/2023 01:00:00', '02/01/2023', image_type='eit171', frequency='12H')
eit195_urls = get_image_urls('01/01/2023 01:13:00', '02/01/2023', image_type='eit195', frequency='12H')
eit284_urls = get_image_urls('01/01/2023 01:06:00', '02/01/2023', image_type='eit284', frequency='12H')
eit304_urls = get_image_urls('01/01/2023 01:19:00', '02/01/2023', image_type='eit304', frequency='12H')



In [17]:
eit304_urls

['https://soho.nascom.nasa.gov/data/REPROCESSING/Completed/2023/eit304/20230101/20230101_0119_eit304_1024.jpg',
 'https://soho.nascom.nasa.gov/data/REPROCESSING/Completed/2023/eit304/20230101/20230101_1319_eit304_1024.jpg',
 'https://soho.nascom.nasa.gov/data/REPROCESSING/Completed/2023/eit304/20230102/20230102_0119_eit304_1024.jpg',
 'https://soho.nascom.nasa.gov/data/REPROCESSING/Completed/2023/eit304/20230102/20230102_1319_eit304_1024.jpg',
 'https://soho.nascom.nasa.gov/data/REPROCESSING/Completed/2023/eit304/20230103/20230103_0119_eit304_1024.jpg',
 'https://soho.nascom.nasa.gov/data/REPROCESSING/Completed/2023/eit304/20230103/20230103_1319_eit304_1024.jpg',
 'https://soho.nascom.nasa.gov/data/REPROCESSING/Completed/2023/eit304/20230104/20230104_0119_eit304_1024.jpg',
 'https://soho.nascom.nasa.gov/data/REPROCESSING/Completed/2023/eit304/20230104/20230104_1319_eit304_1024.jpg',
 'https://soho.nascom.nasa.gov/data/REPROCESSING/Completed/2023/eit304/20230105/20230105_0119_eit304_102

Acá se estan probando el calculo de los parametros que se van implementando para 10 imagenes

In [8]:
import pandas as pd
from urllib.error import HTTPError

def compute_metrics(start_date, end_date, image_type,  frequency) -> pd.DataFrame:
    
    """
    Computes various image metrics for a set of images within a given date range and specified type and frequency.

    Parameters
    ----------
    start_date : str
        Start date for retrieving the images, formatted as 'YYYY-MM-DD'.
    
    end_date : str
        End date for retrieving the images, formatted as 'YYYY-MM-DD'.
    
    image_type : str
        Type/category of the image to retrieve. 
    
    frequency : str
        Frequency for retrieving images, e.g., 'daily', 'hourly'.

    Returns
    -------
    pd.DataFrame
        DataFrame containing computed metrics for each image, with one row per image 
        and columns for each feature. The index of the DataFrame corresponds to the datetime 
        extracted from the image URL.
        
    Notes
    -----
    The function fetches image URLs based on the provided date range, type, and frequency. 
    For each URL, it reads the image and computes a set of metrics, such as entropy, mean 
    intensity, and fractal dimension. If the image at a particular URL is inaccessible, it 
    skips that URL. The extracted date and time from the image URL is used as the index 
    for the resulting DataFrame.
    """

    image_urls = get_image_urls(start_date, end_date, image_type, frequency)
    # Create an empty DataFrame to store the results
    df = pd.DataFrame(columns=['entropy', 'mean_intensity', 'std_deviation', 'fractal_dimension', 'skewness', 'kurtosis', 'uniformity', 'relative_smoothness', 
                                'taruma_contrast', 'taruma_directionality'])

    for url in image_urls:
        try:
            # Try to read the image
            image = io.imread(url)
        except HTTPError:
            #print(f"Unable to access image at {url}")
            continue

        # Compute the features
        entropy = sia.compute_entropy(image)
        mean_intensity = sia.compute_mean_intensity(image)
        std_deviation = sia.compute_standard_deviation(image)
        fractal_dimension = sia.compute_fractal_dimension(image)
        skewness = sia.compute_skewness(image)
        kurtosis = sia.compute_kurtosis(image)
        uniformity = sia.compute_uniformity(image)
        relative_smoothness = sia.compute_relative_smoothness(image)
        taruma_contrast = sia.compute_taruma_contrast(image)
        taruma_directionality = sia.compute_taruma_directionality(image)
        # Extract the date from the URL
        date_string = re.findall(r"\d{8}_\d{4}", url)[0]
        date = datetime.strptime(date_string, '%Y%m%d_%H%M')

        # Append the results to the DataFrame
        df.loc[date] = [entropy, mean_intensity, std_deviation, fractal_dimension, skewness, kurtosis, uniformity, relative_smoothness,taruma_contrast, taruma_directionality ]

    return df

data = compute_metrics('01/01/2023 00:00:00', '01/01/2023 05:00:00','hmiigr','90T')

data
